In [1]:
import glob
import nibabel as nib
import numpy as np
import os
import pandas as pd

# 定義文件目錄路徑
file_pattern = "/NFS/Lulu/MRI/HW2/output_dir/*.nii.gz"
output_dir = "/NFS/Lulu/MRI/HW2"  # 設定輸出資料夾
os.makedirs(output_dir, exist_ok=True)

# 定義標籤名稱和對應數字
labels = {
    1: "Left-Thalamus-Proper", 
    2: "Right-Thalamus-Proper",
    3: "Left-Caudate", 
    4: "Right-Caudate",
    5: "Left-Putamen", 
    6: "Right-Putamen",
    7: "Left-Pallidum", 
    8: "Right-Pallidum",
    9: "Left-Hippocampus", 
    10: "Right-Hippocampus",
    11: "Left-Amygdala", 
    12: "Right-Amygdala"
}

# 初始化 DataFrame 以存儲每個文件和每個結構的體積
final_volume_df = pd.DataFrame()

# 遍歷每個 .nii.gz 文件
for file_path in glob.glob(file_pattern):
    print(f"Processing file: {file_path}")
    
    # 載入影像數據
    img = nib.load(file_path)
    img_data = img.get_fdata()
    
    # 獲取體素尺寸並計算單個體素的體積（以毫米為單位）
    voxel_dims = img.header.get_zooms()
    voxel_volume = np.prod(voxel_dims)  # 單個體素的體積
    
    # 取得原始文件名稱並重新編號 IXI_ID
    base_name = os.path.basename(file_path).replace('.nii.gz', '')
    new_ix_id = base_name[3:6]  # 取第四到第六位數字作為 IXI_ID
    
    # 初始化每個文件的體積字典
    volume_dict = {"IXI_ID": new_ix_id}
    
    # 對每個標籤計算體積
    for label_num, label_name in labels.items():
        # 建立每個標籤的遮罩
        label_mask = (img_data == label_num)
        
        # 計算體素數量並獲得該標籤的總體積
        num_voxels = np.sum(label_mask)
        region_volume = num_voxels * voxel_volume  # 該標籤的總體積
        
        # 將該標籤的體積結果儲存到字典
        volume_dict[label_name] = region_volume
    
    # 將每個文件的結果添加到 DataFrame
    volume_df = pd.DataFrame(volume_dict, index=[0])
    final_volume_df = pd.concat([final_volume_df, volume_df], ignore_index=True)

# 定義 CSV 文件路徑並保存結果
output_csv_path = os.path.join(output_dir, "volume_calculations.csv")
final_volume_df.to_csv(output_csv_path, index=False)
print("Volume calculations saved to:", output_csv_path)

# 顯示計算結果
print(final_volume_df)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI002-Guys-0828-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI012-HH-1211-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI013-HH-1212-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI014-HH-1236-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI015-HH-1258-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI016-Guys-0697-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI017-Guys-0698-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI019-Guys-0702-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI020-Guys-0700-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI021-Guys-0703-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI022-Guys-0701-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI023-Guys-0699-T1_dgm.nii.gz
Processing file: /NFS/Lulu/MRI/HW2/output_dir/IXI024-Guys-0705-T1_dgm.nii.gz
Process

In [2]:
import pandas as pd
import numpy as np

# Paths for the data
volume_csv_path = "/NFS/Lulu/MRI/HW2/segmentation/volume_calculations.csv"
demographics_path = "/NFS/Lulu/MRI/HW2/IXI.csv"

# Load the datasets
volume_df = pd.read_csv(volume_csv_path)
demographics_df = pd.read_csv(demographics_path)

# Merge volume and demographics data on IXI_ID
merged_df = pd.merge(volume_df, demographics_df, on="IXI_ID")

# Dictionary of brain structure labels for analysis
labels = {
    1: "Left-Thalamus-Proper", 
    2: "Right-Thalamus-Proper",
    3: "Left-Caudate", 
    4: "Right-Caudate",
    5: "Left-Putamen", 
    6: "Right-Putamen",
    7: "Left-Pallidum", 
    8: "Right-Pallidum",
    9: "Left-Hippocampus", 
    10: "Right-Hippocampus",
    11: "Left-Amygdala", 
    12: "Right-Amygdala"
}

# Variables for which we want to calculate correlation with brain volumes
variables = ['AGE', 'SEX_ID', 'HEIGHT', 'WEIGHT']

# Initialize a list to store the correlation results
correlation_data = []

# Calculate correlation for each structure and each variable across all patients
for key, structure in labels.items():
    if structure in merged_df.columns:
        # Initialize a dictionary to store correlations for the current structure
        structure_data = {'Structure': structure}
        
        for var in variables:
            # Filter out rows with missing data in the current variable or the structure
            filtered_data = merged_df[[var, structure]].dropna()
            
            if not filtered_data.empty:
                # Calculate the correlation between the variable and the structure
                correlation = filtered_data[var].corr(filtered_data[structure])
                structure_data[f'{var}-Volume Correlation'] = correlation if not np.isnan(correlation) else 'N/A'
            else:
                structure_data[f'{var}-Volume Correlation'] = 'N/A'
        
        # Append the structure's correlation data to the list
        correlation_data.append(structure_data)

# Convert correlation data to DataFrame and display
correlation_report_df = pd.DataFrame(correlation_data)
correlation_report_df.index += 1  # Make index start from 1 instead of 0

# Display the report
print("Correlation Analysis Report Across All Patients:")
print(correlation_report_df)

# Optional: Save the report to a CSV file
output_path = '/NFS/Lulu/MRI/HW2/correlation.csv'
correlation_report_df.to_csv(output_path, index=False)
print(f"Correlation report saved to {output_path}")

Correlation Analysis Report Across All Patients:
                Structure  AGE-Volume Correlation  SEX_ID-Volume Correlation  \
1    Left-Thalamus-Proper               -0.682326                  -0.376038   
2   Right-Thalamus-Proper               -0.657854                  -0.395494   
3            Left-Caudate               -0.490082                  -0.310829   
4           Right-Caudate               -0.495857                  -0.304607   
5            Left-Putamen               -0.507916                  -0.437730   
6           Right-Putamen               -0.498784                  -0.448856   
7           Left-Pallidum               -0.300581                  -0.481569   
8          Right-Pallidum               -0.369204                  -0.437708   
9        Left-Hippocampus               -0.350375                  -0.389509   
10      Right-Hippocampus               -0.430850                  -0.333177   
11          Left-Amygdala               -0.356788                  -0.4